In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import src.config as config

In [4]:
from datetime import datetime
import pandas as pd

current_date = pd.Timestamp('2023-01-01 09:00:00')
#current_date = pd.to_datetime(datetime.utc.now()).floor('H')

# Get pointers to the 2 feature groups we need to create our feature view,
- One with the model predictions,
- The other with the actual demands

In [5]:
from src.feature_store_api import get_feature_group
predictions_fg = get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS
)
actuals_fg = get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.


## Query to generate our feature view from these feature groups

In [6]:
from datetime import timedelta

query = predictions_fg.select_all().join(actuals_fg.select_all(), on=['pickup_hour', 'pickup_location_id']).filter(predictions_fg.pickup_hour >= current_date - timedelta(days=30))

In [7]:
from src.feature_store_api import get_feature_store
feature_store = get_feature_store()
try:
    #create feature view as it does not exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_MONITORING,
        version=1,
        query=query 
    )
except:
    print('Feature view already existed. Skip creation.')

#get feature view 
predictions_and_actuals_fv = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_MONITORING,
    version=1
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.


In [8]:
#fetch predicted and actual values for the last 30 days
preds_and_actuals = predictions_and_actuals_fv.get_batch_data(
    start_time = (current_date - timedelta(days=30)),
    end_time = current_date
)

preds_and_actuals = preds_and_actuals[preds_and_actuals.pickup_hour.between(current_date - timedelta(days=30), current_date)]
preds_and_actuals 

2023-04-16 18:42:55,144 INFO: USE `taxi_batch_scoring_featurestore`
2023-04-16 18:42:56,058 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`pickup_location_id` `pickup_location_id`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_hour` `pickup_hour`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_location_id` `join_pk_pickup_location_id`, `fg1`.`pickup_hour` `join_pk_pickup_hour`, `fg1`.`pickup_hour` `join_evt_pickup_hour`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`predicted_demand` `predicted_demand`, RANK() OVER (PARTITION BY `fg1`.`pickup_location_id`, `fg1`.`pickup_hour`, `fg1`.`pickup_hour` ORDER BY `fg1`.`pickup_hour` DESC) pit_rank_hopsworks
FROM `taxi_batch_scoring_featurestore`.`model_predictions_feature_group_1` `fg1`
INNER JOIN `taxi_batch_scoring_featurestore`.`model_predictions_feature_group_1` `fg1` ON `fg1`.`pickup_location_id` = `fg1`.`pickup_location_id` AND `fg1`.`pickup_hour` = `fg1`.`pickup_hour` AND `fg1`.`pickup_hour` >= `fg1

DatabaseError: Execution failed on sql: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`pickup_location_id` `pickup_location_id`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_hour` `pickup_hour`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_location_id` `join_pk_pickup_location_id`, `fg1`.`pickup_hour` `join_pk_pickup_hour`, `fg1`.`pickup_hour` `join_evt_pickup_hour`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`predicted_demand` `predicted_demand`, RANK() OVER (PARTITION BY `fg1`.`pickup_location_id`, `fg1`.`pickup_hour`, `fg1`.`pickup_hour` ORDER BY `fg1`.`pickup_hour` DESC) pit_rank_hopsworks
FROM `taxi_batch_scoring_featurestore`.`model_predictions_feature_group_1` `fg1`
INNER JOIN `taxi_batch_scoring_featurestore`.`model_predictions_feature_group_1` `fg1` ON `fg1`.`pickup_location_id` = `fg1`.`pickup_location_id` AND `fg1`.`pickup_hour` = `fg1`.`pickup_hour` AND `fg1`.`pickup_hour` >= `fg1`.`pickup_hour`
WHERE `fg1`.`pickup_hour` >= TIMESTAMP '20

In [ ]:
df = predictions_and_actuals
df